In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [ ]:
class Neural_Network():
    def __init__(self, neurons, Activations): 
        # arguments: an array "neurons" consist of number of neurons for each layer, 
        # an array "activations" consisting of activation functions used for the hidden layers and output layer
        self.inputSize = neurons[0] # Number of neurons in input layer
        self.outputSize = neurons[-1] # Number of neurons in output layer
        self.layers = len(neurons)
        self.weights = [] #weights for each layer
        self.biases = [] #biases in each layer 
        self.layer_activations = [] #activations in each layer
        for i in range(len(neurons)-1): 
            self.weights.append(np.random.rand(neurons[i+1],neurons[i])) #weight matrix between layer i and layer i+1
            self.biases.append(np.random.rand(neurons[i+1],1))
            self.layer_activations.append(Activations[i]) #activations for each layer
        
            
    def sigmoid(self, z): # sigmoid activation function
        #Fill in the details to compute and return the sigmoid activation function                  
        return 1.0/(1.0+np.exp(-z))
    
    def sigmoidPrime(self,z): # derivative of sigmoid activation function
        #Fill in the details to compute and return the derivative of sigmoid activation function
        return self.sigmoid(z)*(1-self.sigmoid(z))

                          
    def tanh(self, z): # hyperbolic tan activation function
        #Fill in the details to compute and return the tanh activation function                  
        pass
    
    def tanhPrime(self,x): # derivative of hyperbolic tan activation function
        #Fill in the details to compute and return the derivative of tanh activation function
        pass
                          
    def linear(self, z): # Linear activation function
        #Fill in the details to compute and return the linear activation function                                    
        pass
    
    def linearPrime(self,x): # derivative of linear activation function
        #Fill in the details to compute and return the derivative of activation function                                                      
        pass

    def ReLU(self,z): # ReLU activation function
        #Fill in the details to compute and return the ReLU activation function                  
        pass
    
    def ReLUPrime(self,z): # derivative of ReLU activation function
        #Fill in the details to compute and return the derivative of ReLU activation function
        pass
    
    def forward(self, a): # function of forward pass which will receive input and give the output of final layer
        # Write the forward pass using the weights and biases to find the predicted value and return them.
        layer_activations_a = [a] #store the input as the input layer activations
        layer_dot_prod_z = []
        for i, param in enumerate(zip(self.biases, self.weights)):
            b, w = param[0], param[1]
            if self.layer_activations[i].lower()=='sigmoid':
                z = np.dot(w, a)+b
                a = self.sigmoid(z)
            elif self.layer_activations[i].lower()=='relu':
                pass
            elif self.layer_activations[i].lower()=='tanh':   
                pass
            elif self.layer_activations[i].lower()=='linear':
                pass
            layer_dot_prod_z.append(z)    
            layer_activations_a.append(a)
        return a, layer_dot_prod_z, layer_activations_a
                          
            
    
    def backward(self, x, y, zs, activations): # find the loss and return derivative of loss w.r.t every parameter
        # Write the backpropagation algorithm here to find the gradients of weights and biases and return them.
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        
        # backward pass
        if self.layer_activations[-1].lower()=='sigmoid':
            delta = (activations[-1] - y) * \
                self.sigmoidPrime(zs[-1])
        elif self.layer_activations[-1].lower()=='relu':
            pass
        elif self.layer_activations[-1].lower()=='tanh':   
            pass
        elif self.layer_activations[-1].lower()=='linear':
            pass

        # fill in the appropriate details for gradients of w and b
        #grad_b[-1] = 
        #grad_w[-1] = 
                 
        for l in range(2, self.layers): # Here l is in backward sense i.e. last l th layer
            z = zs[-l]
            if self.layer_activations[-l].lower()=='sigmoid':
                prime = self.sigmoidPrime(z)
            elif self.layer_activations[-l].lower()=='relu':
                pass
            elif self.layer_activations[-l].lower()=='tanh':   
                pass
            elif self.layer_activations[-l].lower()=='linear':
                pass

            #Compute delta, gradients of b and w 
            #delta = 
            #grad_b[-l] = 
            #grad_w[-l] = 
                          
        return (grad_b, grad_w)                 

    def update_parameters(self, grads, learning_rate): # update the parameters using the gradients
        # update weights and biases using the gradients and the learning rate
        
        grad_b, grad_w = grads[0], grads[1]       
        
        #Implement the update rule for weights  and biases
        #self.weights = 
        #self.biases = 
        
    def loss(self, predicted, actual):
        #Implement the loss function
        return 0.5*(predicted-actual)**2
                     
    def train(self, X, Y, minibatch=False): # receive the full training data set
        lr = 1e-3         # learning rate
        epochs = 1000     # number of epochs
        loss_list = []
        if minibatch==False:
            for e in range(epochs): 
                losses = []
                for q in range(len(X)):
                    train_x = np.resize(X[q],(X[q].shape[0],1)) 
                    if not onehotencoded: 
                        train_y = np.resize(Y[q],(1,1)) 
                    else:
                        train_y = np.resize(np.argmax(Y[q]),(1,1)) 
                    out, dot_prod_z, activations_a = self.forward(train_x)
                    loss = self.loss(out, train_y)
                    grads = self.backward(train_x, train_y, dot_prod_z, activations_a) # find the gradients using backward pass
                    self.update_parameters(grads, lr)
                    losses.append(loss)
                loss_list.append(np.mean(np.array(losses)))
                print(f'Epoch: {e} Loss: {np.mean(np.array(losses))}')
        else:
            minibatchsize = 20
            create_minibatches(X,Y,minibatchsize)
            
            for e in range(epochs):
                #Complete the training code with minibatches 
                pass
        return loss_list
        
    def predict(self, x):
        print ("Input : \n" + str(x))
        prediction,_,_ = self.forward(x)
        print ("Output: \n" + str(prediction))


In [ ]:
# a method for creating one hot encoded labels 
def onehotencoding(Y):
    pass

#a method to create minibatches 
def create_minibatches(X,Y,minibatchsize):
    numbatches = int(np.ceil(len(X)/minibatchsize))
    idx = np.arange(len(X))
    np.random.shuffle(idx)
    X_minibatches = []
    Y_minibatches = [] 
    for i in range(numbatches):
        idx_minibatch = idx[i*minibatchsize:min(len(idx),(i+1)*minibatchsize)]
        xn = np.take(X,idx_minibatch,axis=0) 
        yn = np.take(Y,idx_minibatch,axis=0)
        X_minibatches.append(xn)
        Y_minibatches.append(yn)
    return X_minibatches, Y_minibatches

def test_create_minibatches():
    X = []
    Y = []
    inputsize = 3
    minibatch = False
    onehotencoded = False
    n_batch = 20
    batch_size = 5
    for i in range(50):
        if(i % 2 == 0):
            X.append([np.random.randint(1,10) for i1 in range(inputsize)])
            Y.append(1)
        else:
            X.append([np.random.randint(-10,1) for i1 in range(inputsize)])
            Y.append(0)
    X = np.array(X)
    Y = np.array(Y)

    X_mb, Y_mb = create_minibatches(X,Y,6)
    print(X_mb, Y_mb)

#test_create_minibatches()

# Generating some training data

In [ ]:
X = []
Y = []
inputsize = 3
minibatch = False
onehotencoded = False
n_batch = 20
batch_size = 5
for i in range(500):
    if(i % 2 == 0):
        X.append([random.randint(1,10) for i1 in range(inputsize)])
        Y.append(1)
    else:
        X.append([random.randint(-10,1) for i1 in range(inputsize)])
        Y.append(0)
X = np.array(X)
Y = np.array(Y)
if onehotencoded:
    Y = onehotencoding(Y)

if minibatch==False:
    train_X = X
    train_Y = Y
else:
    pass

In [ ]:
print(train_X.shape, train_Y.shape)

# Defining the network

In [ ]:
#D_in is input dimension
#H1 is dimension of first hidden layer 
#H2 is dimension of second hidden layer
#D_out is output dimension.
D_in, H1, H2, D_out = inputsize, 10, 5, 1 #You can add more layers if you wish to 

neurons = [D_in, H1, H2, D_out] # list of number of neurons in the layers sequentially.
activation_functions = ['sigmoid','sigmoid', 'sigmoid'] #activations in each layer (Note: the input layer does not have any activation)
my_neuralnet = Neural_Network(neurons, activation_functions )


# Training the network

In [ ]:
loss = my_neuralnet.train(train_X,train_Y,minibatch=minibatch)

# Prediction for a data point after the training

In [ ]:
my_neuralnet.predict(np.array([8,4,9]).reshape((3,1)))

In [ ]:
plt.plot(loss)
plt.show()